In [ ]:
import pathlib
import os
if 'TSL_SCHOOL_DIR' in os.environ:
    if any( (p/".git").is_dir() for p in (pathlib.Path(".").absolute().resolve()/"dummy").parents ):
        raise RuntimeError('Please copy notebook to a work directory')

# Material definitions in OpenMC

We start by importing the `openmc` module, which contains the API to generate the input files for OpenMC.

In [1]:
import openmc

Materials are created by instantiating a `Material` object:

In [2]:
water = openmc.Material()

After we created the material we can use the `.add_nuclide()` to add nuclides to the composition:

In [3]:
water.add_nuclide('H1', 2.0, 'ao')

The first parameter in `.add_nuclide()` is the nuclide to add, in this case $^1$H, the second is the fraction that it contributes to the composition, and the third parameter is the unit for the fraction. `ao` is a fraction in number of atoms, and `wo` is a fraction in weight. Fraction do not need to add to unity, because compositions are automatically renormalized.

To define the density we use the method `.set_density()`:

In [4]:
water.set_density('g/cm3', 1.0)

Other possible units for the density are atoms per barn*cm (`'atom/b-cm'`) or kg per m$^3$ (`'kg/m3'`).

We can also add elements to the composition with the method `.add_element()`. The element is then separated into its isotopes, based on the available nuclides in the `cross_sections.xml` library. The location of this library is taken from the environmental variable `$OPENMC_CROSS_SECTIONS`, so it is a good practice to set it at the beginning of the script. 

In [5]:
openmc.config['cross_sections'] = '/home/student/openmc_ncrystal_vm/endfb-viii.0-hdf5/cross_sections.xml'
water.add_element('O', 1.0, 'ao')

We can verfiy the composition by listing the nuclide number densities for the material:

In [6]:
water.get_nuclide_atom_densities()

OrderedDict([('H1', 0.06685712956516868),
             ('O16', 0.033349024855540664),
             ('O17', 1.2669426052599466e-05),
             ('O18', 6.687050099108171e-05)])

Materials can also be defined using the `.add_elements_from_formula()` method, which takes a chemical formula and expands it in to the composing nuclides:

In [7]:
poly = openmc.Material()
poly.set_density('g/cc', 0.9)
poly.add_elements_from_formula('CH2')
poly.get_nuclide_atom_densities()

OrderedDict([('C12', 0.038210983660118124),
             ('C13', 0.0004280431388792934),
             ('H1', 0.07726601831392749),
             ('H2', 1.2035284067351716e-05)])

**Although this is handy, if you need to ensure reproducibility, it is better to specify the composition nuclide by nuclide.**

## Thermal nuclear data

The default treatment for the neutron scattering cross section in the thermal range is the free gas model at the temperature of the cell or material. In case the nuclear data library contains an evaluation thermal scattering cross section for the material, these can be specified using the method `.add_s_alpha_beta()`:

In [8]:
water.add_s_alpha_beta("c_H_in_H2O")

Now, the defintition of the material will include the replacement of the scattering cross section for $^1$H with the information in the `c_H_in_H2O` library for thermal neutrons.

## NCrystal materials

NCrystal now also supports the generation of materials where the thermal scattering is handled with NCrystal. These materials are created with the `.from_ncrystal()` function, which takes an NCrystal configuration string and returns an OpenMC material:

In [9]:
cfg = 'MgH2_sg136_MagnesiumHydride.ncmat'
mgh2 = openmc.Material.from_ncrystal(cfg)
mgh2.get_nuclide_densities()

OrderedDict([('H1',
              NuclideTuple(name='H1', percent=0.66656284, percent_type='ao')),
             ('H2',
              NuclideTuple(name='H2', percent=0.00010382666666666666, percent_type='ao')),
             ('Mg24',
              NuclideTuple(name='Mg24', percent=0.26317, percent_type='ao')),
             ('Mg25',
              NuclideTuple(name='Mg25', percent=0.0334, percent_type='ao')),
             ('Mg26',
              NuclideTuple(name='Mg26', percent=0.03676333333333333, percent_type='ao'))])

The composition and density are provided by NCrystal, and the definition includes also the configuration string. During the simulation, OpenMC calls NCrystal to handle the neutron scattering events below 5 eV.

The composition of materials defined from NCrystal cannot be modified after creation.

## Nomenclature of nuclides, elements and compounds

The naming scheme used in OpenMC comes from [GND](https://www.oecd-nea.org/science/wpec/sg38/Meetings/2016_May/tlh4gnd-main.pdf), which is a format that in the future will replace ENDF-6. Instead of using numbers, compounds are defined using strings, with the following criteria:

* Elements: `Sym` (chemical symbol). E.g. `C`, `Al`.
* Nuclides: `SymA` (chemical symbol followd by mass number E.g., `O16`, `Al27`. Nuclides with the average property of the element are represented with mass number zero. E.g.: `C0`.
* Excited states: `SymA_eN` (where N is the number of the excited level). E.g.: `V51_e2`.
* Metastable states: `SymA_mN` (where N is the number of the metastable state). E.g., `Am242_m1`.
* Compounds: `c_name` (where `name` is a string identifying the compound. E.g., `c_H_in_H2O`.

## Temperature

The temperature of the different regions in the simulation is indicated in OpenMC as a property of the cell. But, if all the cells containing a material have the same temperature, this can be indicated with the attribute `.temperature` of the material. The unit for temperature in OpenMC is Kelvin:

In [10]:
water.temperature = 293 # K

If all the system is at the same temperature, this can be specified in the simulation settings (as part of the `Settings()` object). The temperature for the cell has preeminence over the temperature for the material, and the temperature of the material has preeminence over the global value.

## Material collection and generation of xml input

Prior to generating the input it is necessary to generate a collection of materials using the `Materials()` constructor:

In [11]:
mats = openmc.Materials()
mats.append(water)
mats.append(poly)

or, equivalently:

In [12]:
mats = openmc.Materials([water, poly, mgh2])

The generation of the xml input is done using the `.export_to_xml()` method:

In [13]:
mats.export_to_xml()

And we can see the generated file:

In [14]:
!cat materials.xml

<?xml version='1.0' encoding='utf-8'?>
<materials>
  <material id="1" temperature="293">
    <density units="g/cm3" value="1.0" />
    <nuclide ao="2.0" name="H1" />
    <nuclide ao="0.9976206" name="O16" />
    <nuclide ao="0.000379" name="O17" />
    <nuclide ao="0.0020004" name="O18" />
    <sab name="c_H_in_H2O" />
  </material>
  <material id="2">
    <density units="g/cc" value="0.9" />
    <nuclide ao="0.32964066666666664" name="C12" />
    <nuclide ao="0.003692666666666666" name="C13" />
    <nuclide ao="0.66656284" name="H1" />
    <nuclide ao="0.00010382666666666666" name="H2" />
  </material>
  <material cfg="MgH2_sg136_MagnesiumHydride.ncmat" id="3" temperature="293.15">
    <density units="g/cm3" value="1.4185379813679093" />
    <nuclide ao="0.66656284" name="H1" />
    <nuclide ao="0.00010382666666666666" name="H2" />
    <nuclide ao="0.26317" name="Mg24" />
    <nuclide ao="0.0334" name="Mg25" />
    <nuclide ao="0.03676333333333333" name="Mg26" />
  </material>
</mater